In [ ]:
import sqlalchemy
%reload_ext sql
%sql postgresql:///smartprix_laptop_db

In [ ]:
%sql SET search_path TO laptop_schema;

In [19]:
%config SqlMagic.displaycon = False
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

## A. Price and Value Analysis

### Q1: Average price of laptops for each brand
What is the average price of laptops for each brand?

In [20]:
%%sql
SELECT brand, ROUND(AVG(price), 2) AS avg_price
FROM laptops
GROUP BY brand
ORDER BY avg_price DESC;

31 rows affected.

brand,avg_price
Razer,199990.00
Apple,187862.00
Samsung,134224.24
MSI,123260.88
LG,117097.20
Dell,106420.15
Gigabyte,96991.67
Huawei,96830.83
Colorful,89999.00
Microsoft,86794.50


### Q2: Correlation between price and spec score

What is the correlation between price and spec score?

In [21]:
%%sql
SELECT ROUND(CORR(price, spec_score)::NUMERIC, 2) AS correlation
FROM laptops;

1 rows affected.

correlation
0.72


The analysis reveals a strong positive correlation of 0.73 between the `price` and `spec score` of laptops, indicating that higher-priced laptops tend to have higher specification scores.

### Q3: Price difference between NVIDIA and other graphics cards

Is there a significant price difference between laptops with NVIDIA graphics cards and those with other graphics card?

In [22]:
%%sql
SELECT 
    CASE 
        WHEN graphics_card LIKE '%Nvidia%' THEN 'NVIDIA Graphics'
        ELSE 'Other Graphics'
    END AS graphics_type,
    COUNT(*) AS laptop_count,
    ROUND(AVG(price), 2) AS average_price
FROM laptops
GROUP BY 
    CASE 
        WHEN graphics_card LIKE '%Nvidia%' THEN 'NVIDIA Graphics'
        ELSE 'Other Graphics'
    END;

2 rows affected.

graphics_type,laptop_count,average_price
Other Graphics,628,63095.67
NVIDIA Graphics,370,121690.55


Based on the output, there is a substantial price difference between laptops with NVIDIA graphics cards and those with other graphics cards. 

Laptops with NVIDIA graphics have a significantly higher average price (₹122,242.23) compared to those with other graphics cards (₹64,125.52), nearly double the price, suggesting that NVIDIA graphics cards are associated with higher-end, more expensive laptop models.

### Q4: Most common OS in laptops priced over 100,000
Which operating system is most common among laptops priced over 100,000?

In [7]:
%%sql
SELECT os, COUNT(*) AS laptop_count
FROM laptops
WHERE price > 100000
GROUP BY os
ORDER BY laptop_count DESC;

5 rows affected.

os,laptop_count
Windows 11,213
Mac,15
Windows 10,2
Utility: Everyday Use,1
Windows,1


### Q5: Relationship between RAM capacity and price
What is the relationship between RAM capacity and price?

In [8]:
%%sql
WITH ram_extracted AS (
    SELECT 
        price,
        CASE 
            WHEN internal_memory LIKE '%TB%' THEN 
                CAST(SUBSTRING(UPPER(internal_memory) FROM '[0-9]+(?=\s*TB)') AS INTEGER) * 1024
            WHEN internal_memory LIKE '%GB%' THEN 
                CAST(SUBSTRING(UPPER(internal_memory) FROM '[0-9]+(?=\s*GB)') AS INTEGER)
            ELSE 0
        END AS ram_gb
    FROM laptops
),
ram_categories AS (
    SELECT 
        CASE 
            WHEN ram_gb <= 128 THEN '0-128 GB'
            WHEN ram_gb <= 512 THEN '129-512 GB'
            ELSE '513+ GB'
        END AS ram_category,
        price
    FROM ram_extracted
)
SELECT 
    ram_category,
    COUNT(*) AS laptop_count,
    ROUND(AVG(price), 2) AS avg_price,
    MIN(price) AS min_price,
    MAX(price) AS max_price
FROM ram_categories
GROUP BY ram_category
ORDER BY 
    CASE 
        WHEN ram_category = '0-128 GB' THEN 1
        WHEN ram_category = '129-512 GB' THEN 2
        WHEN ram_category = '513+ GB' THEN 3
    END;

3 rows affected.

ram_category,laptop_count,avg_price,min_price,max_price
0-128 GB,23,23709.13,8000,150000
129-512 GB,709,62169.54,15990,231746
513+ GB,266,150473.93,34980,599990


The output shows a clear correlation between RAM capacity and laptop prices, with higher RAM capacities associated with significantly higher average prices. Laptops in the highest RAM category (513+ GB) have an average price more than six times that of the lowest category (0-128 GB), indicating that RAM capacity is a major factor in determining laptop pricing.

### Q6: Average price of laptops with a spec score above 70
What is the average price of laptops with a spec score above 70?

In [9]:
%%sql
SELECT ROUND(AVG(price), 2) AS avg_price_above_70_spec_score
FROM laptops
WHERE spec_score > 70;

1 rows affected.

avg_price_above_70_spec_score
182792.22


## B. Brand and Market Analysis

### Q7: Top 5 laptop brands with highest average user rating
Which are laptop brands have the top 5 highest average user rating?

In [10]:
%%sql
SELECT brand, ROUND(AVG(user_rating), 1) AS avg_user_ratings
FROM laptops
GROUP BY brand
ORDER BY avg_user_ratings DESC
LIMIT 5;

5 rows affected.

brand,avg_user_ratings
Colorful,4.6
Huawei,4.5
LG,4.5
Microsoft,4.4
Tecno,4.4


### Q8: Average warranty period by brand
What is the average warranty period offered by each brand?

In [23]:
%%sql
SELECT brand, 
       ROUND(AVG(CAST(SUBSTRING(warranty FROM '[0-9]+') AS INTEGER)), 2) AS avg_warranty_years
FROM laptops
GROUP BY brand
ORDER BY avg_warranty_years DESC;

31 rows affected.

brand,avg_warranty_years
Gigabyte,2.00
Fujitsu,2.00
MSI,1.80
Xiaomi,1.17
Lenovo,1.11
Huawei,1.00
Ultimus,1.00
Ninkear,1.00
Walker,1.00
Jio,1.00


### Q9: Brand market share across different price segments
How does the market share of different brands vary across different price segments (budget, mid-range, premium)?

In [24]:
%%sql

-- first, create price segments for the laptops
-- Budget: 8,000 - 40,000, Mid-range: 40,001 - 100,000, Premium: 100,001 - 599,990
-- then find the market share, which is:
-- (brand laptop count / total laptop count in price segment) * 100

WITH price_segments_cte AS
(SELECT brand, name, 
        CASE WHEN price <= 40000 THEN 'Budget'
             WHEN price BETWEEN 40001 AND 100000 THEN 'Mid-range'
             ELSE 'Premium' END AS price_segment
 FROM laptops
)
, brand_cte AS (
SELECT brand, price_segment, COUNT(*) AS laptop_count
FROM price_segments_cte
GROUP BY brand, price_segment
)
SELECT price_segment, brand,
       laptop_count,
       SUM(laptop_count) OVER(PARTITION BY price_segment) AS total_in_segment,
       ROUND(100.0 * laptop_count / 
       SUM(laptop_count) OVER(PARTITION BY price_segment), 2) AS market_share_pct
FROM brand_cte
ORDER BY price_segment, market_share_pct DESC;

52 rows affected.

price_segment,brand,laptop_count,total_in_segment,market_share_pct
Budget,Lenovo,42,201,20.90
Budget,HP,34,201,16.92
Budget,Asus,33,201,16.42
Budget,Acer,27,201,13.43
Budget,Dell,16,201,7.96
Budget,Infinix,13,201,6.47
Budget,Chuwi,8,201,3.98
Budget,Zebronics,6,201,2.99
Budget,MSI,4,201,1.99
Budget,Ultimus,4,201,1.99


Summary of the output:

**1. Market Dominance:**

In the budget segment, **Lenovo** leads with **20.98%** market share, followed closely by **HP (17.07%)** and **Asus (16.59%)**. The mid-range segment is also dominated by **Lenovo (22.88%)**, **HP (20.10%)**, and **Asus (19.93%)**, showing their strong presence across both budget and mid-range markets.
In the premium segment, **MSI** takes the lead with **20.59%** market share, closely followed by **HP at 20.17%**, indicating HP's strong presence across all segments.

**2. Brand Diversity:**

The budget segment shows the most brand diversity with 18 different brands represented, including some lesser-known names like **Chuwi, Ultimus, and Primebook**.
The mid-range segment has 20 brands, but with a more concentrated market share among the top brands.
The premium segment is the least diverse with only 12 brands, dominated by well-established names.

**3. Segment Distribution:**

The **mid-range segment** is the largest with **577 laptops**, followed by the **premium segment** with **238 laptops**, and the **budget segment** with **205 laptops**.
Some brands like **Apple** and **Samsung** have a stronger presence in the premium segment compared to budget and mid-range. Certain brands (e.g., **Infinix, Zebronics**) appear in budget and mid-range but not in the premium segment, indicating their market focus.

### Q10: Average user rating in different utility categories
What is the average user rating for laptops in different utility categories (e.g., gaming, business, everyday use) across brands?

In [25]:
%%sql
SELECT utility, ROUND(AVG(user_rating), 2) AS avg_user_rating
FROM laptops
WHERE utility IS NOT NULL
GROUP BY utility
ORDER BY avg_user_rating DESC;

22 rows affected.

utility,avg_user_rating
"Business, Everyday Use, Performance",4.60
"Performance, Business",4.54
"Gaming, Everyday Use, Performance",4.50
"Everyday Use, Gaming, Business, Performance",4.48
"Everyday Use, Gaming, Performance",4.45
"Everyday Use, Performance, Gaming",4.45
"Everyday Use, Business",4.42
"Everyday Use, Business, Performance, Gaming",4.38
"Performance, Everyday Use",4.37
"Gaming, Performance",4.36


### Q11: Brand offering the highest number of Gaming laptops
Which brand offers the highest number of Gaming laptops?

In [14]:
%%sql
SELECT brand, COUNT(*) AS num_of_gaming_laptops
FROM laptops
WHERE utility LIKE '%Gaming%'
GROUP BY brand
ORDER BY num_of_gaming_laptops DESC
LIMIT 1;

1 rows affected.

brand,num_of_gaming_laptops
Lenovo,61


In [26]:
%%sql
WITH relevant_data AS (
    SELECT name, 
           spec_score,
           price,
           ROUND(1.0 * weight / CAST(REGEXP_SUBSTR(screen_size, '[0-9]+(\.[0-9]+)?') AS DECIMAL(3,1)), 2) AS ws_ratio
    FROM laptops
    WHERE weight IS NOT NULL AND screen_size IS NOT NULL
),
lightest_laptops AS (
    SELECT 'Lightest laptop' AS weight_screen_category, 
           name, 
           ws_ratio, 
           price, 
           spec_score
    FROM relevant_data
    ORDER BY ws_ratio ASC
    LIMIT 10
),
heaviest_laptops AS (
    SELECT 'Heaviest laptop' AS weight_screen_category, 
           name, 
           ws_ratio, 
           price, 
           spec_score
    FROM relevant_data
    ORDER BY ws_ratio DESC
    LIMIT 10
)
SELECT *, 
    ROUND(AVG(spec_score) OVER(), 2) AS avg_spec_score,
    ROUND(AVG(price) OVER(), 2) AS avg_price 
FROM lightest_laptops
UNION ALL
SELECT *,
    ROUND(AVG(spec_score) OVER(), 2) AS avg_spec_score,
    ROUND(AVG(price) OVER(), 2) AS avg_price 
FROM heaviest_laptops;

20 rows affected.

weight_screen_category,name,ws_ratio,price,spec_score,avg_spec_score,avg_price
Lightest laptop,Huawei MateBook E 2023 Laptop,0.06,88999,53,62.30,103858.30
Lightest laptop,MSI Prestige 13 AI Evo A1MG Laptop,0.07,99990,66,62.30,103858.30
Lightest laptop,Fujitsu UH-X ‎4ZR1L12856 Laptop,0.07,89990,69,62.30,103858.30
Lightest laptop,MSI Prestige 13 AI Evo A1M Laptop,0.07,113999,64,62.30,103858.30
Lightest laptop,Samsung Galaxy Book2 Pro 13 Laptop,0.07,62990,60,62.30,103858.30
Lightest laptop,Chuwi Ubook X Laptop,0.07,39990,46,62.30,103858.30
Lightest laptop,HP Dragonfly G4 Laptop,0.07,231746,69,62.30,103858.30
Lightest laptop,MSI Prestige 13 Evo A12M-085IN Laptop,0.07,77990,61,62.30,103858.30
Lightest laptop,MSI Prestige 13 AI Evo A1MG 2024 Laptop,0.07,114990,69,62.30,103858.30
Lightest laptop,LG Gram 14 2023 ‎14Z90R-G.CH75A2 Laptop,0.07,117899,66,62.30,103858.30


displaylimit: Value None will be treated as 0 (no limit)